**Version**
** Revision Number **
0  This is just an increasing number. If you make a change increase the number.

**Group members**
Please add your full name and student id(?)
Qinyuan,
Eamon Maloney,
Ibrahim Saana Aminu 25381993,
Des Powell 9513833,
Terence Coffey 15223124

Develop a program in Python using the Keras Neural Network to implement a classifier fot the NMIST handwritten digits database. It is recommended to use a convolution neural network.

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

**Information sources to assist with completing this project goes here.**

[How to create a pdf file for Colab notebook](https://saturncloud.io/blog/convert-google-colab-notebook-to-pdf-html/#:~:text=accessible%20and%20shareable.-,Converting%20Colab%20Notebooks%20to%20PDF,to%20various%20formats%2C%20including%20PDF\)

[Getting started with Keras](https://keras.io/getting_started/)
[Keras for Engineers](https://keras.io/getting_started/intro_to_keras_for_engineers/)

[Alternative solutions](https://en.m.wikipedia.org/wiki/MNIST_database)
[Explanation of some of the layers](https://divingintogeneticsandgenomics.com/post/deep-learning-with-keras-using-mnst-dataset/)





In [ ]:
## Install to allow you to convert this Notebook to a pdf file
! pip install nbconvert
!apt-get install texlive texlive-xetex texlive-latex-extra pandoc

Create the pdf

In [ ]:
!jupyter nbconvert --to pdf Assign1_MNIST_Keras.ipynb

Information about the project
**MNIST character recognition**
Description goes here.

In [1]:
# ------------------------------------------------------
# Required imports
# ------------------------------------------------------
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.regularizers import l2
from keras.callbacks import EarlyStopping
import itertools
import numpy as np
from sklearn.metrics import confusion_matrix
import time


# ------------------------------------------------------
# Constants
# ------------------------------------------------------
NUM_RUNS = 1
NUM_CLASSIFICATIONS = 10
INPUT_SHAPE = (28, 28, 1)
EPOCH_ITERATIONS = 20
VERBOSE_FLAG = 1

# ------------------------------------------------------
# Hyperparameter set of values
# ------------------------------------------------------
model_id_list = [2,3,1]
dropout_rate_list = [0.2, 0.5]
batch_size_list = [32, 64]
val_split_list = [0.2]
learn_rate_list = [0.0001, 0.01]
kernel_size_list = [3, 5]
filter_size_list = [15,32,64]
regularizer_list = [0.000001]
seed_value_list = [42]

# ------------------------------------------------------
# currently not being used
# ------------------------------------------------------
early_stopping = EarlyStopping(monitor='val_loss', patience=6)
fit_callbacks = [early_stopping]


# ------------------------------------------------------
# create the specific set of models to be explored
# ------------------------------------------------------
def create_model(model_id, filter_size, kernel_size, dropout_rate, regularizer, seed_value):
    tf.random.set_seed(seed_value)

    # Define model structures based on model_id
    if model_id == 1:
        model = keras.Sequential([
            layers.Input(shape=INPUT_SHAPE),
            layers.Conv2D(filter_size, kernel_size=(kernel_size, kernel_size), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Conv2D(filter_size * 2, kernel_size=(kernel_size, kernel_size), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Flatten(),
            layers.Dropout(dropout_rate),
            layers.Dense(10, activation="softmax")
        ], name='Model1')
        model_description = "Conv2D with two Conv layers, two MaxPool, Dropout, and Dense output."
    
    elif model_id == 2:
        model = keras.Sequential([
            layers.Input(shape=INPUT_SHAPE),
            layers.Conv2D(filter_size, kernel_size=(kernel_size, kernel_size), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Dropout(dropout_rate), 
            layers.Conv2D(filter_size * 2, kernel_size=(kernel_size, kernel_size), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Dropout(dropout_rate), 
            layers.Flatten(),
            layers.Dense(filter_size * 4, activation="relu"),
            layers.Dense(10, activation="softmax")
        ], name='Model2')
        model_description = "Conv2D with two of Conv,MaxPool,Dropout layers, with one internal Dense layer followed by Dense output."

    elif model_id == 3:
        model = keras.Sequential([
            layers.Input(shape=INPUT_SHAPE),
            layers.Conv2D(filter_size, kernel_size=(kernel_size, kernel_size), padding='same', activation="relu", 
                          kernel_initializer=HeNormal(seed=seed_value)),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Conv2D(filter_size, kernel_size=(kernel_size, kernel_size), padding='same', activation="relu", 
                          kernel_initializer=HeNormal(seed=seed_value), 
                          kernel_regularizer=regularizers.l2(regularizer)),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Flatten(),
            layers.Dropout(dropout_rate),
            layers.Dense(256, activation="relu", kernel_regularizer=regularizers.l2(regularizer)),   
            layers.Dropout(dropout_rate),
            layers.Dense(10, activation="softmax")
        ], name='Model3')
        model_description = "Structure based on Lecture notes detail"
    
    else:
        raise ValueError("Invalid model_id. Please use 1, 2, or 3.")
    
    return model, model_description


# ------------------------------------------------------
# Load and preprocess dataset
# ------------------------------------------------------
def load_and_preprocess_data():

    # Load and pre-process MNIST dataset
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
    x_train = x_train.astype("float32") / 255.0
    x_test = x_test.astype("float32") / 255.0
    y_train = keras.utils.to_categorical(y_train, num_classes=NUM_CLASSIFICATIONS)
    y_test = keras.utils.to_categorical(y_test, num_classes=NUM_CLASSIFICATIONS)
    
    return (x_train, y_train), (x_test, y_test)

# ------------------------------------------------------
# Compile and train the model
# ------------------------------------------------------
def compile_and_train_model(model, x_train, y_train, batch_size, learn_rate, val_split):
    # compile the model
    model.compile(
        loss="categorical_crossentropy",
        optimizer=Adam(learning_rate=learn_rate),
        metrics=['accuracy', Precision(), Recall()]
    )
    # train the model
    history = model.fit(x_train, y_train, batch_size=batch_size, epochs=EPOCH_ITERATIONS, validation_split=val_split,shuffle=True,verbose=VERBOSE_FLAG) #, callbacks=fit_callbacks)     
    
    return history

# ------------------------------------------------------
# Run experiments
# ------------------------------------------------------
def run_experiments():
    (x_train, y_train), (x_test, y_test) = load_and_preprocess_data()
    
    # Example for running experiments with different model structures

    experiment_count=0
    for params in itertools.product(model_id_list,dropout_rate_list, batch_size_list, val_split_list, learn_rate_list, kernel_size_list, filter_size_list, regularizer_list, seed_value_list):
        model_id, dropout_rate, batch_size, val_split, learn_rate, kernel_size, filter_size, regularizer, seed_value = params     
        experiment_count+=1
        # Create model with model description
        model, model_description = create_model(model_id,filter_size, kernel_size, dropout_rate, regularizer, seed_value)
        # Train and evaluate
        history = compile_and_train_model(model, x_train, y_train, batch_size, learn_rate, val_split)         
        # predict using the test data
        predictions = model.predict(x_test)
        predicted_labels = np.argmax(predictions, axis=1)
        true_labels = np.argmax(y_test, axis=1)
        conf_matrix = confusion_matrix(true_labels, predicted_labels)    
        
                                
        results_model = model.evaluate(x_test, y_test, verbose=VERBOSE_FLAG)
        num_epochs_run = early_stopping.stopped_epoch if early_stopping.stopped_epoch > 0 else EPOCH_ITERATIONS
        print(f"{model.name} - Description: {model_description}")
        print((
            f"#Epochs: {num_epochs_run}, Dropout Rate: {dropout_rate}, Batch Size: {batch_size}, Validation Split: {val_split}, "
            f"Learning Rate: {learn_rate}, Kernel Size: {kernel_size}, Filter Size: {filter_size}, Regularizer: {regularizer}, Seed: {seed_value}, "
            f"ACCURACY: {results_model[1]}, Loss: {results_model[0]}, Precision: {results_model[2]}, Recall: {results_model[3]}, Experiment #{experiment_count}"
        ))
        print(f'Confusion Matrix\n{conf_matrix}\nModel Summary\n{model.summary()}')
                    
# ------------------------------------------------------
# main
# ------------------------------------------------------
# go for it...
for number_of_runs in range(NUM_RUNS):
    start_time = time.time()
    print(f'Start Run #{number_of_runs+1} of {NUM_RUNS}')
    run_experiments()
    end_time = time.time()
    print(f'Finished Run #{number_of_runs+1} of {NUM_RUNS}, duration {end_time-start_time} seconds')



2024-10-30 21:48:10.653051: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-30 21:48:10.739378: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730324890.777786    8604 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730324890.789071    8604 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-30 21:48:10.871553: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Start Run #1 of 1


W0000 00:00:1730324894.091799    8604 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.5830 - loss: 1.3028 - precision: 0.8062 - recall: 0.3586 - val_accuracy: 0.9320 - val_loss: 0.2406 - val_precision: 0.9558 - val_recall: 0.9139
Epoch 2/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9104 - loss: 0.2971 - precision: 0.9339 - recall: 0.8900 - val_accuracy: 0.9553 - val_loss: 0.1607 - val_precision: 0.9680 - val_recall: 0.9449
Epoch 3/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9371 - loss: 0.2069 - precision: 0.9509 - recall: 0.9240 - val_accuracy: 0.9641 - val_loss: 0.1261 - val_precision: 0.9724 - val_recall: 0.9578
Epoch 4/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9509 - loss: 0.1627 - precision: 0.9603 - recall: 0.9424 - val_accuracy: 0.9684 - val_loss: 0.1071 - val_precision: 0.9756 - val_recall: 0.9627
Epoch 5/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9575 - loss: 0.1400 - precision: 0.9651 - recall: 0.9505 - val_accuracy: 0.9728 - va

Model: "Model2"

...

 Total params: 681,248 (2.60 MB)

 Trainable params: 227,082 (887.04 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 454,166 (1.73 MB)

Confusion Matrix
[[ 977    0    1    0    0    0    1    1    0    0]
 [   4 1120    2    1    0    0    5    2    1    0]
 [   3    1 1025    0    1    0    0    2    0    0]
 [   0    1    0  998    0    4    0    1    5    1]
 [   0    0    0    0  969    0    1    1    2    9]
 [   6    0    0    5    0  873    3    0    2    3]
 [   5    1    0    0    2    1  948    0    1    0]
 [   0    5   15    0    1    0    0 1000    2    5]
 [   6    0    2    0    0    2    2    0  960    2]
 [   2    3    0    0    9    4    0    5   10  976]]
Model Summary
None
Finished Run #1 of 1, duration 49696.59025979042 seconds


Application using Keras for MNIST Digit Classifier using 2 convolution layers , 2 drop out layers and 2 accumulation layers. Split the dataset into training and test data in the ratio 70 percent and 30 percent respectively.

Display accuracy of Testing and Validation dataset

Discuss the Project especially
- How you choose to tackle it
- What design decisions you made
- What the results are like
- What you might do better/differently next time you had to tackle a similar project
- If plots are called for they should be in your code and in your report.
- Marks for neat well designed code with appropriate level of comments
- neat logically laid out and informative reports.
- Provide classification accuracy for the training and test data. The test data should be split in the ration 70 to 80 and the baance for validation.


